In [37]:
import torch
device = 'cpu'

In [6]:
with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))

42460
﻿The Project Gutenbe
['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [54]:
char_to_int = dict()
int_to_char = dict()
for i,c in enumerate(chars):
    char_to_int[c] = i
    int_to_char[i] = c
encoder = lambda s: torch.tensor([char_to_int[c] for c in s])
decoder = lambda l: ''.join([int_to_char[i] for i in l])

In [35]:
tensor = torch.tensor(encoder(text), dtype=torch.long)
n = int(0.8 * len(tensor))
data = tensor[:n]
eval = tensor[n:]
print(data[:9])

tensor([90, 47, 64, 61,  1, 43, 74, 71, 66])


In [119]:
block_size = 8
batch_size = 4
def get_batch(split):
    if split == "training":
        training_data = data
    else:
        training_data = eval
    ix = torch.randint(len(training_data) - block_size - 1, (batch_size,))
    x = torch.stack([ training_data[i:i+block_size] for i in ix])
    y = torch.stack([ training_data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x,y = get_batch("training")

In [132]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

class LLM(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_table = torch.nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets):
        # B, T, C
        logits = self.token_table(index)
        if targets is None:
            return logits, None
        B,T,C = logits.size()
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = torch.nn.functional.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_gen_token=500):
        for i in range(max_gen_token):
            logits, loss = self.forward(index, None)
            logits = logits[-1,]
            prob = torch.nn.functional.softmax(logits, dim=-1)
            index_next = torch.multinomial(prob, num_samples=1)
            index = torch.cat((index, index_next), dim=0)
        return index

    def train_and_update(self, get_batch, lr, epoch):
        optimizer = torch.optim.AdamW(self.parameters(), lr=lr)
        for _ in range(int(epoch)):
            x, y = get_batch("training")
            logits, loss = self.forward(x, y)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

In [133]:
print("vocab size is %d" %(len(chars)))
llm = LLM(len(chars))

vocab size is 91


In [ ]:
context = encoder("hello")
gen = llm.generate(context)
llm.to(device)
print(decoder(gen.tolist()))
llm.train_and_update(get_batch, 3e-4, 1e30)